In [ ]:
%load_ext autoreload
%autoreload 2
#Assessments with confirmed active results
#107617, 107614

# 17 mins for current operation with all ids
# 2 mins for 100 ids. 

#To DO
#Implement logging to the whole operations

from modules.auth import *
from modules.assessments_endpoints import *
from modules.frame_transformations import *
from modules.config import base_url_illuminate
from airflow import DAG
from airflow.operators.python import PythonOperator
import logging
import os

os.makedirs('logs', exist_ok=True)
logging.basicConfig(filename='./logs/Illuminate_operations.log', level=logging.INFO,
                   format='%(asctime)s - %(message)s', datefmt='%d-%b-%y %H:%M:%S', force=True)
logging.info('\n\n-------------New Illuminate Operations Logging Instance')


access_token, expires_in = get_access_token()

#For testing
# assessment_id_list = assessment_id_list[0:10]
temp = get_all_assessments_metadata(access_token)
assessment_id_list = list(temp['assessment_id'].unique())

test_results_group, log_results_group = loop_through_assessment_scores(access_token, assessment_id_list, 'Group') #Seperate table
test_results_standard, log_results_standard = loop_through_assessment_scores(access_token, assessment_id_list, 'Standard')
test_results_no_standard, log_results_no_standard = loop_through_assessment_scores(access_token, assessment_id_list, 'No_Standard')

test_results_combined = bring_together_test_results(test_results_no_standard, test_results_standard)
test_results_view = create_test_results_view(test_results_combined, access_token, '23-24') #add in grade level col, string matching

#Write dfs to SFTP
#Takes 4 mins with current 167 test_ids. 

#takes 25 mins to run all 

#41000 rows to now 58000 rows

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The length of the ID_list is 1077
The length of the ID_list is 1077
The length of the ID_list is 1077


In [38]:
test_results_view

,assessment_id,year,date_taken,grade,local_student_id,test_type,curriculum,unit,title,standard_code,score,performance_band_level,performance_band_label,proficiency,mastered,__count,last_update
8007,107502,23-24,2024-07-03,3,601989,checkpoint,IM,Unit 1,3rd Grade - IM Unit 1 - Introducing Multiplica...,CCSS.Math.Content.3.MD.B.3,75,2,Approaching Expectations,2 Approaching Expectations,f,410,2024-11-05
7977,107502,23-24,2024-07-03,3,600893,checkpoint,IM,Unit 1,3rd Grade - IM Unit 1 - Introducing Multiplica...,CCSS.Math.Content.3.MD.B.3,100,4,Exceeds Expectations,4 Exceeds Expectations,t,410,2024-11-05
7978,107502,23-24,2024-07-03,3,600344,checkpoint,IM,Unit 1,3rd Grade - IM Unit 1 - Introducing Multiplica...,CCSS.Math.Content.3.MD.B.3,62,2,Approaching Expectations,2 Approaching Expectations,f,410,2024-11-05
7979,107502,23-24,2024-07-03,3,600216,checkpoint,IM,Unit 1,3rd Grade - IM Unit 1 - Introducing Multiplica...,CCSS.Math.Content.3.OA.D.8,0,1,Does Not Meet Expectations,1 Does Not Meet Expectations,f,410,2024-11-05
7980,107502,23-24,2024-07-03,3,601768,checkpoint,IM,Unit 1,3rd Grade - IM Unit 1 - Introducing Multiplica...,CCSS.Math.Content.3.OA.D.8,0,1,Does Not Meet Expectations,1 Does Not Meet Expectations,f,410,2024-11-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4121,115910,23-24,2024-11-05,1,601008,checkpoint,IM,Unit 2,1st Grade - IM Unit 2 - Section C Checkpoint,CCSS.Math.Content.1.MD.C.4,100,4,Exceeds Expectations,4 Exceeds Expectations,t,660,2024-11-05
4122,115910,23-24,2024-11-05,1,602035,checkpoint,IM,Unit 2,1st Grade - IM Unit 2 - Section C Checkpoint,CCSS.Math.Content.1.OA.C.6,100,4,Exceeds Expectations,4 Exceeds Expectations,t,660,2024-11-05
4123,115910,23-24,2024-11-05,1,601580,checkpoint,IM,Unit 2,1st Grade - IM Unit 2 - Section C Checkpoint,CCSS.Math.Content.1.OA.C.6,100,4,Exceeds Expectations,4 Exceeds Expectations,t,660,2024-11-05
4111,115910,23-24,2024-11-05,1,601403,checkpoint,IM,Unit 2,1st Grade - IM Unit 2 - Section C Checkpoint,CCSS.Math.Content.1.OA.B.4,80,3,Meets Expectations,3 Meets Expectations,t,660,2024-11-05


# TO DO

Changes for the test_results_combined frame
Now represents all assessments standard & no standard. 

Both frames brought together, and added in standard_code column. 
All values in 'standard_code' column with a percent value are no_standard values in the 'Standard_No_Standard' column as well. So if you were looking to fitler on this. Looks like you can do it either way

Changes for the test_results_view
asssesment_id column is now present
values in kinder column that were 'K' are now 0

General notes
The date taken filter on all of these assessment pulls is from 2024-07-01 up to the current run day
